# MFPT Optimisation Benchmarking

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time

from tqdm import tqdm

import openmm
from openmm import unit
from openmm.app.topology import Topology
from openmm.app.element import Element
import mdtraj
import csv

import config
from dham import *
from util import apply_fes, try_and_optim_M, apply_bias, update_bias, get_total_bias,try_and_optim_sparse_M
from scipy.sparse import csr_matrix

In [2]:
def propagate(simulation,
              gaussian_params,
              prop_index, 
              pos_traj,   #this records the trajectory of the particle. in shape: [prop_index, sim_steps, 3]
              steps=config.propagation_step,
              dcdfreq=config.dcdfreq,
              stepsize=config.stepsize,
              num_bins=config.num_bins,
              pbc=config.pbc,
              time_tag=None,
              top=None,
              reach=None
              ):
    """
    here we use the openmm context object to propagate the system.
    save the CV and append it into the CV_total.
    use the DHAM_it to process CV_total, get the partially observed Markov matrix from trajectory.
    return the current position, the CV_total, and the partially observed Markov matrix.
    """
    directory_paths = ["./trajectory/explore","./figs/explore","./params","./visited_states"]
    for i in directory_paths:
        if not os.path.exists(i):
            os.makedirs(i)
    file_handle = open(f"./trajectory/explore/{time_tag}_langevin_sim_explore_{prop_index}.dcd", 'bw')
    
    dcd_file = openmm.app.dcdfile.DCDFile(file_handle, top, dt = stepsize) #note top is no longer a global pararm, we need pass this.
    for _ in tqdm(range(int(steps/dcdfreq)), desc=f"Propagation {prop_index}"):
        simulation.integrator.step(dcdfreq)
        state = simulation.context.getState(getPositions=True, enforcePeriodicBox=pbc)
        dcd_file.writeModel(state.getPositions(asNumpy=True))
    file_handle.close()

    #save the top to pdb.
    with open(f"./trajectory/explore/{time_tag}_langevin_sim_explore_{prop_index}.pdb", 'w') as f:
        openmm.app.PDBFile.writeFile(simulation.topology, state.getPositions(), f)
    
    #we load the pdb and pass it to mdtraj_top
    mdtraj_top = mdtraj.load(f"./trajectory/explore/{time_tag}_langevin_sim_explore_{prop_index}.pdb")

    #use mdtraj to get the coordinate of the particle.
    traj = mdtraj.load_dcd(f"./trajectory/explore/{time_tag}_langevin_sim_explore_{prop_index}.dcd", top = mdtraj_top)#top = mdtraj.Topology.from_openmm(top)) #this will yield error because we using imaginary element X.
    coor = traj.xyz[:,0,:] #[all_frames,particle_index,xyz] # we grep the particle 0.

    #we digitize the x, y coordinate into meshgrid (0, 2pi, num_bins)
    x = np.linspace(0, 2*np.pi, num_bins) #hardcoded.

    #we digitize the coor into the meshgrid.
    coor_x = coor.squeeze()[:,:1] #we only take the xcoordinate.
    #we test.
    if False: 
        plt.figure()
        plt.plot(coor_x)
        plt.xlim([0, 2*np.pi])
        plt.savefig("./test.png")
        plt.close()
    #we append the coor_xy_digitized into the pos_traj.
    pos_traj[prop_index,:] = coor_x.squeeze()

    #we take all previous digitized x and feed it into DHAM.
    coor_x_total = pos_traj[:prop_index+1,:].squeeze() #note this is in coordinate space np.linspace(0, 2*np.pi, num_bins)

    #here we use the DHAM.
    F_M, MM = DHAM_it(coor_x_total.reshape(-1,1), gaussian_params, T=300, lagtime=1, num_bins=num_bins, time_tag=time_tag, prop_index=prop_index)
    cur_pos = coor_x_total[-1] #the current position of the particle, in ravelled 1D form.
    

    #determine if the particle has reached the target state.
    end_state_xyz = config.end_state.value_in_unit_system(openmm.unit.md_unit_system)[0]
    end_state_x = end_state_xyz[:1]
    for index_d, d in enumerate(coor_x):
        #if the distance of current pos is the config.target_state, we set reach to index_d.
        target_distance = np.linalg.norm(d - end_state_x)
        if target_distance < 0.1:
            reach = index_d * config.dcdfreq

    return cur_pos, pos_traj, MM, reach, F_M

def get_working_MM(M):
    zero_rows = np.where(~M.any(axis=1))[0]
    zero_cols = np.where(~M.any(axis=0))[0]

    keep_indices = np.setdiff1d(range(M.shape[0]), np.union1d(zero_rows, zero_cols))
    M_work = M[np.ix_(keep_indices, keep_indices)]
    return M_work, keep_indices

def find_closest_index(working_indices, final_index, N):
    """
    returns the farest index in 1D.

    here we find the closest state to the final state.
    first we unravel all the index to 2D.
    then we use the lowest RMSD distance to find the closest state.
    then we ravel it back to 1D.
    note: for now we only find the first-encounted closest state.
          we can create a list of all the closest states, and then choose random one.
    """
    def rmsd_dist(a, b):
        return np.sqrt(np.sum((a-b)**2))
    working_x, working_y = np.unravel_index(working_indices, (N,N), order='C')
    working_states = np.stack((working_x, working_y), axis=1)
    final_state = np.unravel_index(final_index, (N,N), order='C')
    closest_state = working_states[0]
    for i in range(len(working_states)):
        if rmsd_dist(working_states[i], final_state) < rmsd_dist(closest_state, final_state):
            closest_state = working_states[i]
        
    closest_index = np.ravel_multi_index(closest_state, (N,N), order='C')
    return closest_index

def DHAM_it(CV, gaussian_params, T=300, lagtime=2, num_bins=150, prop_index=0, time_tag=None):
    """
    intput:
    CV: the collective variable we are interested in. now it's 2d.
    gaussian_params: the parameters of bias potential. (in our case the 10-gaussian params)
     format: (a,bx, by,cx,cy)
    T: temperature 300

    output:
    the Markov Matrix
    Free energy surface probed by DHAM.
    """
    d = DHAM(gaussian_params, num_bins=num_bins)
    d.setup(CV, T, prop_index=prop_index, time_tag=time_tag)

    d.lagtime = lagtime
    d.num_bins = num_bins #num of bins, arbitrary.
    results = d.run(biased = True, plot=True)
    return results

def random_initial_bias(initial_position, num_gaussians):
    #returns random a,b,c for 10 gaussian functions. around the initial position
    # initial position is in Anstrom
    initial_position = initial_position.value_in_unit_system(openmm.unit.md_unit_system)[0] #this is in nm
    rng = np.random.default_rng()
    #a = np.ones(10)
    a = np.ones(num_gaussians) * 0.01 * 4.184 #convert to kJ/mol
    b = rng.uniform(initial_position[0]-0.5, initial_position[0]+0.5, num_gaussians)
    c = rng.uniform(0, 2*np.pi, num_gaussians)
    return np.concatenate((a,b,c), axis=None)


In [14]:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])  # Example matrix A

# Extract the diagonal of A
diag_A = np.diag(A)

# Subtract the diagonal from each column of A
B =  diag_A[ None,:]-A

In [21]:
diag_A[ None,:] 

array([[1, 5, 9]])

In [18]:
1/3 * 4

1.3333333333333333

In [15]:
B

array([[ 0,  3,  6],
       [-3,  0,  3],
       [-6, -3,  0]])

In [3]:
matrix = np.zeros((5,5)) + np.identity(5) + 1

In [4]:
matrix = matrix - 2*np.ones(5)

In [35]:
New = csr_matrix(matrix)

In [87]:
New[4][0][0].toarray()[0][1]

-1.0

In [9]:
from scipy.sparse import linalg

In [12]:
%%time
linalg.inv(MM_sparse.astype('float64'))

/home/gsaba/miniconda3/envs/biophys_env/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)


RuntimeError: Factor is exactly singular

In [40]:
New.toarray()

array([[ 0.        , -1.        , -1.        , -1.        , -1.        ],
       [-1.        ,  0.        , -1.        , -1.        , -1.        ],
       [-0.33333333, -0.33333333,  0.        , -0.33333333, -0.33333333],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-1.        , -1.        , -1.        , -1.        ,  0.        ]])

In [90]:
(New[1] * 2).toarray(),(New[1]).toarray()

(array([[-2.,  0., -2., -2., -2.]]), array([[-1.,  0., -1., -1., -1.]]))

In [5]:
print(reach)

None


In [24]:
np.where(matrix.any(axis=1))[0]

array([0, 1, 2, 4])

In [21]:
get_working_MM(matrix)

(array([[ 0., -1., -1., -1.],
        [-1.,  0., -1., -1.],
        [-1., -1.,  0., -1.],
        [-1., -1., -1.,  0.]]),
 array([0, 1, 2, 4]))

In [3]:
if __name__ == "__main__":
    elem = Element(0, "X", "X", 1.0)
    top = Topology()
    top.addChain()
    top.addResidue("xxx", top._chains[0])
    top.addAtom("X", elem, top._chains[0]._residues[0])
    mass = 12.0 * unit.amu
    for i_sim in range(1):
    #def simulate_once():
        print("system initializing")
        #print out all the config.
        print("config: ", config.__dict__)
        
        time_tag = time.strftime("%Y%m%d-%H%M%S")

        #print current time tag.
        print("time_tag: ", time_tag)

        system = openmm.System() #we initialize the system every
        system.addParticle(mass)
        #gaussian_param = np.loadtxt("./params/gaussian_fes_param.txt")
        system, fes = apply_fes(system = system, 
                                particle_idx=0, 
                                gaussian_param = None, 
                                pbc = config.pbc, 
                                amp = config.amp, 
                                name = "FES",
                                mode=config.fes_mode, 
                                plot = True)
        y_pot = openmm.CustomExternalForce("1e3 * y^2") # very large force constant in y
        y_pot.addParticle(0)
        z_pot = openmm.CustomExternalForce("1e3 * z^2") # very large force constant in z
        z_pot.addParticle(0)
        system.addForce(z_pot) #on z, large barrier
        system.addForce(y_pot) #on y, large barrier

        #pbc section
        if config.pbc:
            a = unit.Quantity((2*np.pi*unit.nanometers, 0*unit.nanometers, 0*unit.nanometers))
            b = unit.Quantity((0*unit.nanometers, 2*np.pi*unit.nanometers, 0*unit.nanometers))
            c = unit.Quantity((0*unit.nanometers, 0*unit.nanometers, 1*unit.nanometers)) # atom not moving in z so we set it to 1 nm
            system.setDefaultPeriodicBoxVectors(a,b,c)

        #integrator
        integrator = openmm.LangevinIntegrator(300*unit.kelvin, 
                                            1.0/unit.picoseconds, 
                                            0.002*unit.picoseconds)

        num_propagation = int(config.sim_steps/config.propagation_step)
        frame_per_propagation = int(config.propagation_step/config.dcdfreq)
        #this stores the digitized, ravelled, x, y coordinates of the particle, for every propagation.
        pos_traj = np.zeros([num_propagation, frame_per_propagation]) #shape: [num_propagation, frame_per_propagation]

        x = np.linspace(0, 2*np.pi, config.num_bins)

        #we start propagation.
        #note num_propagation = config.sim_steps/config.propagation_step
        reach = None
        i_prop = 0
        #for i_prop in range(num_propagation):
        while reach is None:
            if i_prop >= num_propagation:
                print("propagation number exceeds num_propagation, break")
                break
            if i_prop == 0:
                print("propagation 0 starting")
                gaussian_params = random_initial_bias(initial_position = config.start_state, num_gaussians = config.num_gaussian)
                
                #we apply the initial gaussian bias (v small) to the system
                system = apply_bias(system = system, particle_idx=0, gaussian_param = gaussian_params, pbc = config.pbc, name = "BIAS", num_gaussians = config.num_gaussian)

                #create simulation object, this create a context object automatically.
                # when we need to pass a context object, we can pass simulation instead.
                simulation = openmm.app.Simulation(top, system, integrator, config.platform)
                simulation.context.setPositions(config.start_state)
                simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

                simulation.minimizeEnergy()
                if config.pbc:
                    simulation.context.setPeriodicBoxVectors(a,b,c)

                #now we propagate the system, i.e. run the langevin simulation.
                cur_pos, pos_traj, MM, reach, F_M = propagate(simulation = simulation,
                                                                    gaussian_params = gaussian_params,
                                                                    prop_index = i_prop,
                                                                    pos_traj = pos_traj,
                                                                    steps=config.propagation_step,
                                                                    dcdfreq=config.dcdfreq,
                                                                    stepsize=config.stepsize,
                                                                    num_bins=config.num_bins,
                                                                    pbc=config.pbc,
                                                                    time_tag = time_tag,
                                                                    top=top,
                                                                    reach=reach
                                                                    )

                #working_MM, working_indices = get_working_MM(MM) no longer needed as sparse matrix data structures are more efficient
                MM_sparse = config.sparse_mat[0](MM)
                working_indices = np.intersect1d(np.unique(MM_sparse.row),np.unique(MM_sparse.col))
                MM_sparse = MM_sparse.tocsr()
                final_coor = config.end_state.value_in_unit_system(openmm.unit.md_unit_system)[0][:1]
                final_index = np.digitize(final_coor, x)
                closest_index = working_indices[np.argmin(np.abs(working_indices - final_index))] #find the closest index in working_indices to final_index.
                i_prop += 1
            else:

                print(f"propagation number {i_prop} starting")

                #find the most visited state in last propagation.
                last_traj = pos_traj[i_prop-1,:]
                last_traj_index = np.digitize(last_traj, x).astype(np.int64)
                most_visited_state = np.argmax(np.bincount(last_traj_index)) #this is in digitized

                #gaussian_params 
                gaussian_params= try_and_optim_sparse_M(MM_sparse,
                                                         working_indices,
                                                         num_gaussian = config.num_gaussian,
                                                         start_index = most_visited_state,
                                                         end_index = closest_index,
                                                         plot = False,
                                                         )
                
                # gaussian_params = try_and_optim_M(working_MM,
                #                                 working_indices = working_indices,
                #                                 num_gaussian = config.num_gaussian,
                #                                 start_index = most_visited_state,
                #                                 end_index = closest_index,
                #                                 plot = False,
                #                                 )
                if True:
                    #here we calculate the total bias given the optimized gaussian_params
                    total_bias = get_total_bias(x, gaussian_params, num_gaussians=config.num_gaussian) * 4.184 #convert to kcal/mol
                    plt.figure()
                    plt.plot(x, total_bias, label="total bias applied")
                    #plt.savefig(f"./figs/explore/{time_tag}_total_bias_{i_prop}.png")
                    #plt.close()
                    
                    #here we plot the reconstructed fes from MM.
                    # we also plot the most_visited_state and closest_index.
                    #plt.figure()
                    plt.plot(x, F_M*4.184, label="DHAM fes")
                    plt.plot(x[most_visited_state], F_M[most_visited_state]*4.184, marker='o', markersize=3, color="blue", label = "most visited state (local start)")
                    plt.plot(x[closest_index], F_M[closest_index]*4.184, marker='o', markersize=3, color="red", label = "closest state (local target)")
                    #plt.legend()
                    #plt.savefig(f"./figs/explore/{time_tag}_reconstructed_fes_{i_prop}.png")
                    #plt.close()

                    #we plot here to check the original fes, total_bias and trajectory.
                
                    #we add the total bias to the fes.
                    #fes += total_bias_big
                    #plt.figure()
                    plt.plot(x, fes, label="original fes")
                    plt.xlabel("x-coor position (nm)")
                    #plt.xlim([-1, 2*np.pi+1])
                    #plt.ylim([-1, 2*np.pi+1])
                    plt.ylabel("fes (kJ/mol)")
                    plt.title("FES mode = multiwell, pbc=False")
                    
                    #additionally we plot the trajectory.
                    # first we process the pos_traj into x, y coordinate.
                    # we plot all, but highlight the last prop_step points with higher alpha.

                    history_traj = pos_traj[:i_prop, :].squeeze() #note this is only the x coor.
                    recent_traj = pos_traj[i_prop:, :].squeeze()
                    #here we digitize this so we can plot it to fes.
                    history_traj = np.digitize(history_traj, x)
                    recent_traj = np.digitize(recent_traj, x)

                    plt.scatter(x[history_traj], fes[history_traj], s=3.5, alpha=0.3, c='grey')
                    plt.scatter(x[recent_traj], fes[recent_traj], s=3.5, alpha=0.8, c='black')
                    plt.legend(loc='upper left')
                    plt.savefig(f"./figs/explore/{time_tag}_fes_traj_{i_prop}.png")
                    plt.close()


                #save the gaussian_params
                np.savetxt(f"./params/{time_tag}_gaussian_fes_param_{i_prop}.txt", gaussian_params)

                #apply the gaussian_params to openmm system.
                simulation = update_bias(simulation = simulation,
                                        gaussian_param = gaussian_params,
                                        name = "BIAS",
                                        num_gaussians=config.num_gaussian,
                                        )
                
                #we propagate system again
                cur_pos, pos_traj, MM, reach, F_M = propagate(simulation = simulation,
                                                                    gaussian_params = gaussian_params,
                                                                    prop_index = i_prop,
                                                                    pos_traj = pos_traj,
                                                                    steps=config.propagation_step,
                                                                    dcdfreq=config.dcdfreq,
                                                                    stepsize=config.stepsize,
                                                                    num_bins=config.num_bins,
                                                                    pbc=config.pbc,
                                                                    time_tag = time_tag,
                                                                    top=top,
                                                                    reach=reach
                                                                    )
                #update working_MM and working_indices
                MM_sparse = config.sparse_mat[0](MM)
                working_indices = np.intersect1d(np.unique(MM_sparse.row),np.unique(MM_sparse.col))
                MM_sparse = MM_sparse.tocsr()
                #update closest_index
                closest_index = working_indices[np.argmin(np.abs(working_indices - final_index))] #find the closest index in working_indices to final_index.
                
                i_prop += 1

        #we have reached target state, thus we record the steps used.
        total_steps = i_prop * config.propagation_step + reach * config.dcdfreq
        print("total steps used: ", total_steps)

        with open("./total_steps_mfpt.csv", "a") as f:
            writer = csv.writer(f)
            writer.writerow([total_steps])

        #save the pos_traj
        np.savetxt(f"./visited_states/{time_tag}_pos_traj.txt", pos_traj)

    """from multiprocessing import Pool
    
    multi_process_result = []
    for _ in range(config.num_sim//config.NUMBER_OF_PROCESSES):
        with Pool(config.NUMBER_OF_PROCESSES) as p:
            multi_process_result.extend(p.map(simulate_once, range(config.NUMBER_OF_PROCESSES)))
"""
print("all done")


system initializing
config:  {'__name__': 'config', '__doc__': None, '__package__': '', '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0x7f9d20d0aed0>, '__spec__': ModuleSpec(name='config', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7f9d20d0aed0>, origin='/home/gsaba/MarkovMastersProject/RL_MFPT/1D_langevin_sim/config.py'), '__file__': '/home/gsaba/MarkovMastersProject/RL_MFPT/1D_langevin_sim/config.py', '__cached__': '/home/gsaba/MarkovMastersProject/RL_MFPT/1D_langevin_sim/__pycache__/config.cpython-311.pyc', '__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, types, exceptions, and other objects.\n\nThis module provides direct access to all 'built-in'\nidentifiers of Python; for example, builtins.len is\nthe full name for the built-in function len().\n\nThis module is not normally accessed explicitly by most\napplications, but can be useful in modules that provide\nobjects with the same name as a built-in value, but in\

Propagation 0: 100%|██████████| 50/50 [00:00<00:00, 175.27it/s]
/home/gsaba/MarkovMastersProject/RL_MFPT/1D_langevin_sim/util.py:145: RuntimeWarning: divide by zero encountered in log
  F = -kT * np.log(peq)
/home/gsaba/MarkovMastersProject/RL_MFPT/1D_langevin_sim/util.py:131: RuntimeWarning: divide by zero encountered in log
  F = -kT * np.log(peq)
/home/gsaba/miniconda3/envs/biophys_env/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/home/gsaba/miniconda3/envs/biophys_env/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:302: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


sum of peq in dham reconstruction:  0.99999999999999999995
propagation number 1 starting
optimizing to get g_param from start state: 3 to end state: 11 in working indices.
converted to xspace that's from: 2.199114857512855 to 2.7017696820872223
upper bound: 2.7017696820872223 lower bound: 2.0106192982974678
random try: 0 mfpt: 16.45269894246529
Performing Kemeny constant check...
the min/max of the Kemeny constant is: 16.57146803812304 19.07145682949882
random try: 100 mfpt: 21.495322743263703
Performing Kemeny constant check...
the min/max of the Kemeny constant is: 17.37523261786314 19.356665967554274
random try: 200 mfpt: 18.764416411400305
Performing Kemeny constant check...
the min/max of the Kemeny constant is: 16.933608832381996 19.2571481283586
random try: 300 mfpt: 22.554231009913178
Performing Kemeny constant check...
the min/max of the Kemeny constant is: 17.494292520836776 19.34212568958388
random try: 400 mfpt: 24.211515396519935
Performing Kemeny constant check...
the min

Propagation 1: 100%|██████████| 50/50 [00:00<00:00, 104.27it/s]


sum of peq in dham reconstruction:  1.0000000000000000002
propagation number 2 starting
optimizing to get g_param from start state: 5 to end state: 15 in working indices.
converted to xspace that's from: 2.0734511513692637 to 2.7017696820872223
upper bound: 2.7017696820872223 lower bound: 1.6964600329384885
random try: 0 mfpt: 27.254772042192485
Performing Kemeny constant check...
the min/max of the Kemeny constant is: 28.112130839944527 30.92502612387528
random try: 100 mfpt: 39.98982637990343
Performing Kemeny constant check...
the min/max of the Kemeny constant is: 28.723391858171276 31.069009074091912
random try: 200 mfpt: 39.93122034047859
Performing Kemeny constant check...
the min/max of the Kemeny constant is: 28.54246491575661 30.764403467016127
random try: 300 mfpt: 20.570859876053298
Performing Kemeny constant check...
the min/max of the Kemeny constant is: 27.04853928638675 30.35136870348624
random try: 400 mfpt: 32.97401932490135
Performing Kemeny constant check...
the min

...

Propagation 98: 100%|██████████| 50/50 [00:00<00:00, 88.65it/s]


sum of peq in dham reconstruction:  1.0
propagation number 99 starting
optimizing to get g_param from start state: 13 to end state: 23 in working indices.
converted to xspace that's from: 2.1362830044410597 to 2.7646015351590183
upper bound: 2.7646015351590183 lower bound: 1.3194689145077132
random try: 0 mfpt: 28.424912837841656
Performing Kemeny constant check...
the min/max of the Kemeny constant is: 43.128378005088436 44.74703675305745
random try: 100 mfpt: 29.1524696124674
Performing Kemeny constant check...
the min/max of the Kemeny constant is: 43.77677861828621 45.24375256165042
random try: 200 mfpt: 40.60305447781489
Performing Kemeny constant check...
the min/max of the Kemeny constant is: 44.93425067524986 46.190114064903696
random try: 300 mfpt: 59.65714528506167
Performing Kemeny constant check...
the min/max of the Kemeny constant is: 44.73713929084156 46.0926660822266
random try: 400 mfpt: 73.13910995837466
Performing Kemeny constant check...
the min/max of the Kemeny co

Propagation 99: 100%|██████████| 50/50 [00:00<00:00, 109.53it/s]


sum of peq in dham reconstruction:  1.0
propagation number exceeds num_propagation, break


TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

In [5]:
i_prop

100

In [21]:
def Markov_mfpt_calc_sparse(peq, M):
    N = M.shape[0]
    onevec = np.ones(N)
    Idn = eye(N)  # Sparse identity matrix
    print("peq: ",peq)
    print("peq reshaped (-1,1)",peq.reshape(-1,1))
    A = config.sparse_mat[1](((peq.reshape(-1, 1)) @ onevec.T).T)
    print(M.data)
    print(Idn + A - M)
    Qinv = linalg.inv(Idn + A - M)  # Sparse matrix inversion

    mfpt = np.zeros((N, N))
    for j in range(N):
        for i in range(N):
            term1 = Qinv[j, j] - Qinv[i, j] + Idn[i, j]
            if peq[j] * term1 == 0:
                mfpt[i, j] = 1e12  # Use a large number instead of explicit value
            else:
                mfpt[i, j] = 1/peq[j] * term1
    return mfpt

In [33]:
from scipy.linalg import inv

In [32]:
%%time
linalg.inv((Idn + A - M).astype('float64'))

CPU times: user 10.8 ms, sys: 0 ns, total: 10.8 ms
Wall time: 10.4 ms


/home/gsaba/miniconda3/envs/biophys_env/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/home/gsaba/miniconda3/envs/biophys_env/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:302: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


<100x100 sparse matrix of type '<class 'numpy.float64'>'
	with 2476 stored elements in Compressed Sparse Row format>

In [35]:
Q_dense = (Idn + A - M).astype('float64').toarray()

In [36]:
%%time
inv(Q_dense)

CPU times: user 853 ms, sys: 360 ms, total: 1.21 s
Wall time: 82.2 ms


array([[ 1., -0.,  0., ..., -0.,  0., -0.],
       [ 0.,  1.,  0., ..., -0.,  0., -0.],
       [ 0.,  0.,  1., ..., -0.,  0., -0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0., -0.],
       [ 0.,  0.,  0., ...,  0.,  1., -0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [24]:
Idn.shape

(100, 100)

In [29]:
peq = peq[:100]

In [26]:
A.shape

(100, 108)

In [30]:

N = MM_sparse.shape[0]
onevec = np.ones(N)
Idn = eye(N)  # Sparse identity matrix
peq_new = config.sparse_mat[1](peq.reshape(-1, 1))

print(peq_new,onevec.T)
A = (peq_new.multiply(onevec.T)).T
#Qinv = linalg.inv((Idn + A - M).astype('float64')) # Sparse matrix inversion

# Qinv_diag = Qinv.diagonal()
# peq_matrix = np.tile(peq, (N, 1))
# peq_nonzero_mask = peq_matrix != 0
# mfpt_dense = np.full((N, N), fill_value=1e12)
# term1_matrix = np.tile(Qinv_diag, (N, 1)) - Qinv.toarray()
# np.fill_diagonal(term1_matrix, Qinv_diag + Idn.diagonal())
# mfpt_dense[peq_nonzero_mask] = term1_matrix[peq_nonzero_mask] / peq_matrix[peq_nonzero_mask]
# mfpt_optimized_sparse = config.sparse_mat[1](mfpt_dense)


# for j in range(N):
#     for i in range(N):
#         term1 = Qinv[j, j] - Qinv[i, j] + Idn[i, j]
#         if peq[j] * term1 == 0:
#             mfpt[i, j] = 1e12  # Use a large number instead of explicit value
#         else:
#             mfpt[i, j] = 1/peq[j] * term1


  (28, 0)	0.06067021
  (29, 0)	0.06146722
  (30, 0)	0.0622529
  (31, 0)	0.0630296
  (32, 0)	0.06380288
  (33, 0)	0.06458124
  (34, 0)	0.06537591
  (35, 0)	0.06620055
  (36, 0)	0.06707084
  (37, 0)	0.06800414
  (38, 0)	0.06901899
  (39, 0)	0.07013462
  (40, 0)	0.07137035
  (41, 0)	0.0727449
  (42, 0)	0.07427566 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]


In [36]:
mfpt_optimized_sparse.toarray()[40]

array([1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       3.62188324e+01, 3.46966066e+01, 4.48058018e+01, 5.47956375e+01,
       4.44540330e+01, 4.67232899e+01, 4.88219649e+01, 3.17381696e+01,
       2.44102012e+01, 1.46883066e+01, 6.63215695e+00, 1.08430610e+01,
       3.81758771e+01, 2.74186962e+01, 2.31730459e+01, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
      

In [11]:
from scipy.sparse import diags, eye
from scipy.sparse import linalg,diags

In [12]:
Markov_mfpt_calc_sparse(peq, bias_mat)

peq:  [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.06067021 0.06146722
 0.0622529  0.0630296  0.06380288 0.06458124 0.06537591 0.06620055
 0.06707084 0.06800414 0.06901899 0.07013462 0.07137035 0.0727449
 0.07427566 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.  

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 100 is different from 1)

In [4]:
MM_sparse.data

array([1.        , 0.50259762, 0.49740238, 1.        , 0.20071176,
       0.40029615, 0.19969948, 0.19929261, 0.14321546, 0.57157787,
       0.28520667, 0.08391362, 0.1669808 , 0.49992177, 0.16627136,
       0.08291246, 0.20057046, 0.4002521 , 0.39917744, 0.16790454,
       0.16753251, 0.16708269, 0.16652821, 0.16586175, 0.16509031,
       0.50306022, 0.49693978, 0.67048231, 0.32951769, 0.50116548,
       0.49883452, 0.33642846, 0.66357154], dtype=float128)

In [6]:
print(csr_matrix(np.zeros((5,5))).data)

[]


In [4]:
print(MM_sparse)

  (29, 29)	0.3332803572911920194
  (29, 30)	0.6667196427088079806
  (30, 29)	0.33335530786434554548
  (30, 31)	0.6666446921356544545
  (31, 29)	0.25042258794774740468
  (31, 30)	0.2504822962261473101
  (31, 33)	0.24980056624349338352
  (31, 34)	0.24929454958261190167
  (32, 31)	1.0
  (33, 33)	0.33378409375424774843
  (33, 34)	0.66621590624575225157
  (34, 32)	0.2011740994518226312
  (34, 33)	0.20087721366960539249
  (34, 35)	0.19995818209381306999
  (34, 36)	0.19934691037578208208
  (34, 37)	0.19864359440897682424
  (35, 34)	0.40085887530288522926
  (35, 35)	0.39983484787837184025
  (35, 36)	0.19930627681874293048
  (36, 34)	0.25130545563339104994
  (36, 35)	0.2506634748906570818
  (36, 37)	0.49803106947595186826
  (37, 35)	0.16797158512862434545
  (37, 36)	0.33491619273278013137
  (37, 38)	0.33241171407365481302
  (37, 40)	0.16470050806494071014
  (38, 39)	1.0
  (39, 37)	0.3354526787922488724
  (39, 39)	0.49899891592191072395
  (39, 40)	0.16554840528584040366
  (40, 37)	0.252809188566

In [7]:
csr_matrix(([1,2,3,4,5,0,7],[0,1,2,3,4,5,0,2],[0,4,4]),shape = (5,5))

ValueError: index pointer size (3) should be (6)